In [1]:
# !pip install pymysql

In [1]:
import pymysql
import os
from datetime import date

In [2]:
host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
database = os.getenv('MYSQL_DATABASE')

In [3]:
conn = pymysql.connect(
    host=host,
    port=int(3306),
    user="root",
    passwd="Ram@77793",
#     db="project1",
    charset='utf8mb4')

In [4]:
print(conn)

In [5]:
my_cursor=conn.cursor()

In [6]:
my_cursor.execute("SHOW DATABASES")

11

In [7]:
for i in my_cursor:
    print(i)

('abc',)
('information_schema',)
('john',)
('major',)
('mini',)
('mysql',)
('performance_schema',)
('project',)
('project1',)
('ram',)
('sys',)


In [8]:
my_cursor.execute("USE PROJECT1")

0

In [9]:
my_cursor.execute("SHOW TABLES")

4

In [10]:
for i in my_cursor:
    print(i)

('bills',)
('items',)
('login',)
('transaction',)


In [11]:
def clear():  
    for i in range(2):
        print()

In [12]:
def login():
    while True:
        clear()
        uname=input('enter your id:')
        upass=input('enter your password')
        my_cursor.execute('select *from login where name="{}"and pwd="{}"'.format(uname,upass)) #  it will take values according to format.
        my_cursor.fetchall()
        rows=my_cursor.rowcount
        if rows!=1:
            print('invalid login details....try again')
        else:
            print('you have logged in successfully...')
            print('\n\n')
            print('Press any key to continue.....')
            break

In [14]:
def last_bill_no():
    my_cursor.execute('select max(bill_id) from bills')
    record = my_cursor.fetchone()
    return record

In [15]:
def find_item(no):
    my_cursor.execute('select * from items where id ={}'.format(no))
    record = my_cursor.fetchone()
    return record

In [17]:
def add_item():
    clear()
    print('Add New Item - Screen')
    print('-'*100)
    item_name = input('Enter new Item Name :')  
    item_price = input('Enter Item Price :')
    sql = 'select * from items where item_name like "%{}%"'.format(item_name)
    my_cursor.execute(sql)
    record=my_cursor.fetchone()
    if record==None:
        sql = 'insert into items(item_name,price) values("{}",{});'.format(item_name,item_price)
        my_cursor.execute(sql)
        print('\n\nNew Item added successfully.....\nPress any key to continue....')
    else:
        print('\n\nItem Name already Exist.....\nPress any key to continue....')
    wait= input()  

In [18]:
#   function name       : modify_item
#   purpose             : change item details in items table
def modify_item():
    clear() 
    print('Modify Item Details -')
    print('-'*100)
    item_id = input('Enter Item ID :')
    item_name = input('Enter new Item Name :')
    item_price = input('Enter Item Price :')
    sql = 'update items set item_name = "{}", price ={} where id={}'.format(item_name,item_price,item_id)
    my_cursor.execute(sql)
    print('\n\nRecord Updated Successfully............')
    wait = input()

In [19]:
#   function name           : item_list
#   purpose                 : To display all the items in items tables
def item_list():
    clear()
    sql="select * from items"
    my_cursor.execute(sql)
    records = my_cursor.fetchall()
    for row in records:
        print(row)
    print('\nPress any key to continue.....')
    wait=input()

In [20]:
#   function name       : billing
#   purpose             : To generate bills 
def billing():
    clear()
    items =[]
    bill_no = last_bill_no()
    if bill_no[0]==None:
        bill_no =1
    else:
        bill_no = bill_no[0]+1 

    name = input('Enter customer Name :')
    phone = input('Enter Phone No :')
    today = date.today()
    while True:
        no = int(input('Enter item No (0 to stop) :'))
        if no <=0:
            break
        else:
            item = find_item(no)
            if item==None:
                print('Item Not found  ')
            else:
                qty = int(input('Enter Item Qty :'))
                item = list(item)
                item.append(qty)
                items.append(item)

    clear()
    print('                      Green Valley Supermarket              ')
    print('                           Retail Invoice                   ')
    print('                     Phone: 9939928284, Email: greenvalley@supermarket.com ')
    print('Bill No :{}                           Date :{}'.format(bill_no,today))
    print('-'*100)
    print('Customer Name :{}                     Phone No :{}'.format(name,phone))
    print('-'*100)
    print('Item Id        Item Name                Price            Qty         Amount ')
    print('-'*100)
    total =0
    for item in items:
        print('{:<10d} {:25s} {:.2f} {:>10d}               {:>.2f} \
            '.format(item[0],item[1],item[2],item[3],item[2]*item[3]))
        total = total +item[2]*item[3]
    print('-'*100)
    print('Total Payable amount : {}'.format(total))
    print('\nPress any key to continue........')
    #insert data into tables
    sql ='insert into bills(name,phone,bill_date) values("{}","{}","{}");'.format(name,phone,today)
    my_cursor.execute(sql)    
    for item in items:
        sql='insert into transaction(item_id,qty,bill_id) values({},{},{});'.format(item[0],item[3],bill_no)
        my_cursor.execute(sql)
    wait = input()

In [21]:
#   function      : Date_wise_sell
#   purpose       : Create a report on date wise sell or sell between two dates 
def date_wise_sell():
    clear()
    print('Sell Between Two Dates -- Screen')
    print('-'*100)
    start_date = input('Enter start Date (yyyy-mm-dd) :')
    end_date   = input('Enter End Date (yyyy-mm-dd) :')
    sql = 'select * from bills where bill_date between "{}" and "{}"'.format(start_date,end_date)
    my_cursor.execute(sql)
    records = my_cursor.fetchall()
    clear()
    print('Bill No       Customer Name         Phone No         Bill Date')
    print('-'*100)
    for row in records:
        print('{:10d} {:30s} {:20s} {}'.format(row[0],row[1],row[2],row[3]))
    print('-'*100)
    print('\n\nPress any key to continue....')
    wait= input()

In [22]:
# function name        : bill information
# purpose               : display details of any bill
def bill_information():
    clear()
    bill_no = input('Enter Bill Number :')
    sql = 'select b.bill_id,b.name,b.phone,b.bill_date,t.item_id,t.qty,i.item_name,i.price from bills b,transaction t,items i \
           where b.bill_id = t.bill_id AND t.item_id= i.id AND \
           b.bill_id ={};'.format(bill_no)
    my_cursor.execute(sql) 
    records = my_cursor.fetchall()
    n = my_cursor.rowcount
    clear()
    print("Bill No :",bill_no)
    print('-'*100)
    if n<=0:
        print('Bill number {} does not exists'.format(bill_no))
    else:
        print('Customer Name : {}  Phone No :{}'.format(records[0][1],records[0][2]))
        print('Bill Date : {}'.format(records[0][3]))
        print('-'*100)
        print('{:10s} {:30s} {:20s} {:10s} {:30s}'.format('ID','Item Name','Qty','Price','Amount'))
        print('-'*100)
        for row in records:
            print('{:<10d} {:30s} {:<20d} {:.2f} {:>.2f}'.format(row[4],row[6],row[5],row[7],row[5]*row[7]))
        print('-'*100)
    print('\nPress any key to continue....')
    wait = input()   

In [23]:
#  function name    : amount_collected
#  purpose          : Function to display amount collected between two dates
def amount_collected():
    clear()
    start_date = input('Enter start Date (yyyy-mm-dd) :')
    end_date   = input('Enter End   Date (yyyy-mm-dd) :')
    clear()
    print('Amount collected between: {} and {}'.format(start_date,end_date))
    print('-'*100)
    sql = 'select sum(t.qty*i.price) from bills b,transaction t,items i \
           where b.bill_date between "{}" AND "{}" AND b.bill_id = t.bill_id AND \
           t.item_id = i.id;'.format(start_date,end_date)
    my_cursor.execute(sql)
    result = my_cursor.fetchone()
    print(result)
    print('\nPress any key to continue.....')
    wait= input()

In [24]:
def search_item():
    clear()
    item_name =input('Enter Item Name :')
    sql ='select * from items where item_name like "%{}%";'.format(item_name)
    my_cursor.execute(sql)
    records = my_cursor.fetchall()
    clear()
    print('Item Names start with :',item_name)
    print('-'*100)
    print('{:10s} {:30s} {:20s}'.format('Item ID','Item Name','Item Price'))
    print('-'*100)
    for row in records:
        print('{:<10d} {:30s} {:.2f}'.format(row[0],row[1],row[2]))
    print('-'*100)
    print('\nPress any key to continue....')

In [25]:
def search_customer():
    clear()
    cust_name =input('Enter customer Name :')
    sql ='select * from bills where name like "%{}%";'.format(cust_name)
    my_cursor.execute(sql)
    records = my_cursor.fetchall()
    clear()
    print('Customer Names started with :',cust_name)
    print('-'*100)
    print('{:10s} {:30s} {:20s} {:20s}'.format('Bill No','Customer Name','Phone No','Bill Date'))
    print('-'*100)
    for row in records:
        print('{:<10d} {:30s} {:20s} {:20s}'.format(row[0],row[1],row[2],str(row[3])))
    print('-'*100)
    print('\nPress any key to continue....')
    wait= input()

In [26]:
# function name      : search_bill
# purpose            : function to find out bill information
def search_bill():
    clear()
    bill_no = input('Enter Bill Number :')
    sql = 'select b.bill_id,b.name,b.phone,b.bill_date,t.item_id,t.qty,i.item_name,i.price from bills b,transaction t,items i \
           where b.bill_id = t.bill_id AND t.item_id= i.id AND \
           b.bill_id ={};'.format(bill_no)
    my_cursor.execute(sql) 
    records = my_cursor.fetchall()
    n = my_cursor.rowcount
    clear()
    print("Bill No :",bill_no)
    print('-'*100)
    if n<=0:
        print('Bill number {} does not exists'.format(bill_no))
    else:
        print('Customer Name : {}  Phone No :{}'.format(records[0][1],records[0][2]))
        print('Bill Date : {}'.format(records[0][3]))
        print('-'*100)
        print('{:10s} {:30s} {:20s} {:10s} {:30s}'.format('ID','Item Name','Qty','Price','Amount'))
        print('-'*100)
        for row in records:
            print('{:<10d} {:30s} {:<20d} {:.2f} {:>.2f}'.format(row[4],row[6],row[5],row[7],row[5]*row[7]))
        print('-'*100)
    print('\nPress any key to continue....')

In [27]:
#  function name    : search_menu
#  purpose          : Display search menu on the screen
def search_menu():
    while True:
        clear()
        print('      S E A R C H    M E N U ')
        print('-'*100)
        print('1.   Item Name')
        print('2.   Customer information')
        print('3.   Bill information')
        print('4.   Back to main Menu')
        choice = int(input('\n\nEnter your choice (1..4): '))
        if choice==1:
            search_item()
        if choice==2:
            search_customer()
        if choice==3:
            search_bill()
        if choice==4:
            break

In [28]:
#  function name    : report_menu
#  purpose          : Display report menu on the screen
def report_menu():
    while True:
        clear()
        print('   R E P O R T   M E N U ')
        print('-'*100)
        print('1.   Item List')
        print('2.   Sell Between Dates')
        print('3.   Bill information')
        print('4.   Amount collected')
        print('5.   Back to main Menu')
        choice = int(input('\n\nEnter your choice (1..5): '))
        if choice==1:
            item_list()
        if choice==2:
            date_wise_sell()
        if choice==3:
            bill_information()
        if choice==4:
            amount_collected()
        if choice==5:
            break

In [29]:
def main_menu():
    while True:
        #clear()
        print('      M A I N   M E N U')
        print('-'*100)
        print('1.   Add New Item')
        print('2.   Modify Existing Item')
        print('3.   Billing')
        print('4.   Search Menu')
        print('5.   Report Menu')
        print('6.   Exit')
        choice = int(input('\n\nEnter your choice (1..6): '))
        if choice==1:
            add_item()
        if choice==2:
            modify_item()
        if choice==3:
            billing()
        if choice==4:
            search_menu()
        if choice==5:
            report_menu()
        if choice==6:
            break

In [30]:
if __name__=="__main__":
    clear()
    login()
    main_menu()





enter your id:Ram
enter your password1234
you have logged in successfully...



Press any key to continue.....
      M A I N   M E N U
----------------------------------------------------------------------------------------------------
1.   Add New Item
2.   Modify Existing Item
3.   Billing
4.   Search Menu
5.   Report Menu
6.   Exit


Enter your choice (1..6): 3


Enter customer Name :Ram
Enter Phone No :9533
Enter item No (0 to stop) :1
Enter Item Qty :10
Enter item No (0 to stop) :2
Enter Item Qty :12
Enter item No (0 to stop) :0


                      Green Valley Supermarket              
                           Retail Invoice                   
                     Phone: 9939928284, Email: greenvalley@supermarket.com 
Bill No :1                           Date :2023-01-03
----------------------------------------------------------------------------------------------------
Customer Name :Ram                     Phone No :9533
-----------------------------------------------